In [ ]:
import warnings
warnings.filterwarnings("ignore")

import string
import keras
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from plotly import graph_objs as go
from plotly import express as px
from plotly import figure_factory as ff
from collections import Counter
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
! pip install bs4
from bs4 import BeautifulSoup

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

import tensorflow as tf
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.preprocessing import text, sequence
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU
from keras.callbacks import ReduceLROnPlateau

from string import punctuation
from nltk.tokenize.toktok import ToktokTokenizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [ ]:
primary_blue = "#496595";primary_blue2 = "#85a1c1";primary_blue3 = "#3f4d63";primary_grey = "#c6ccd8";

# Data Exploration

In [ ]:
sms = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv", encoding='latin-1')
sms

In [ ]:
sms.dropna(how="any", inplace=True, axis=1)
sms.columns = ['label', 'message']
sms.head()

In [ ]:
sms.describe()

In [ ]:
sms.groupby('label').describe()

In [ ]:
sms['messagelen'] = sms.message.apply(len)
sms.head()

In [ ]:
sms[sms.label=='ham'].describe()

In [ ]:
sms[sms.label=='spam'].describe()

In [ ]:
balance_counts = sms.groupby('label')['label'].agg('count').values
balance_counts

# Data Visulaization

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=['ham'],
    y=[balance_counts[0]],
    name='ham',
    text=[balance_counts[0]],
    textposition='auto',
    marker_color=primary_blue
))
fig.add_trace(go.Bar(
    x=['spam'],
    y=[balance_counts[1]],
    name='spam',
    text=[balance_counts[1]],
    textposition='auto',
    marker_color=primary_grey
))
fig.update_layout(
    title='<span style="font-size:32px; font-family:Times New Roman">Dataset distribution by target</span>'
)
fig.show()

In [ ]:
plt.figure(figsize=(12,5))
sms[sms['label']=='spam']['messagelen'].plot(bins=35,kind='hist',color='blue',label='spam',alpha=0.5)
plt.legend()
plt.xlabel('message length')
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
sms[sms['label']=='ham']['messagelen'].plot(bins=35,kind='hist',color='red',label='spam',alpha=0.5)
plt.legend()
plt.xlabel('message length')
plt.show()

In [ ]:
df=sms.copy()
df.label.replace("ham",0,inplace = True)
df.label.replace("spam",1,inplace = True)
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

# Data Cleaning Indepth 

In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            if i.strip().isalpha():
                final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_stopwords(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
df['message']=df['message'].apply(denoise_text)

# EDA

In [ ]:
plt.figure(figsize = (20,20)) # Text that is not Spam
wc = WordCloud(max_words = 2000 , width = 1600 , height = 800 , stopwords = STOPWORDS).generate(" ".join(df[df.label == 0].message))
plt.imshow(wc , interpolation = 'bilinear')

In [ ]:
plt.figure(figsize = (20,20)) # Text that is Spam
wc = WordCloud(max_words = 2000 , width = 1600 , height = 800 , stopwords = STOPWORDS).generate(" ".join(df[df.label == 1].message))
plt.imshow(wc , interpolation = 'bilinear')

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(20,10))
word=df[df['label']==1]['message'].str.split().apply(lambda x : [len(i) for i in x])
sns.distplot(word.map(lambda x: np.mean(x)),ax=ax1,color='red')
ax1.set_title('Spam')
word=df[df['label']==0]['message'].str.split().apply(lambda x : [len(i) for i in x])
sns.distplot(word.map(lambda x: np.mean(x)),ax=ax2,color='green')
ax2.set_title('Not Spam')
fig.suptitle('Average word length in each text')

**lets perform stemming or lemmetzation and vectorization** 

In [ ]:
df1=df.copy()

In [ ]:
ps = PorterStemmer() # Using porterstemmer for text preprocessing
message = []
for i in range(0, df1.shape[0]):
    review = re.sub('[^a-zA-Z]', ' ', df1['message'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    message.append(review)

In [ ]:
df1['clean_msg']=np.empty((len(message),1))
for i in range(len(message)):
    df1['clean_msg'][i]=message[i]
df1['clean_msg_len']=df1['clean_msg'].apply(len)
df1.head()

In [ ]:
X=df1['clean_msg']
Y=df1['label']

In [ ]:
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(message).toarray()
X

In [ ]:
Y=np.array(Y)
type(Y)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.10, random_state = 0)


In [ ]:
print("X_train shape: {}\n X_test shape: {}\nY_train shape: {}\nY_test shape: {}".format(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape))

In [ ]:
acc=[]

# Ml algorithms

In [ ]:
spam_detect_model = MultinomialNB().fit(X_train, Y_train)
pred=spam_detect_model.predict(X_test)

print("Accuracy of Naive Bayes Classifier is: {}".format(metrics.accuracy_score(Y_test,pred)))
acc.append(metrics.accuracy_score(Y_test,pred))

In [ ]:
LR=LogisticRegression(solver='liblinear')
LR.fit(X_train,Y_train)
yhat = LR.predict(X_test)
print("LogisticRegression's Accuracy:{0}".format(metrics.accuracy_score(Y_test, yhat)))
acc.append(metrics.accuracy_score(Y_test,yhat))

In [ ]:
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, Y_train) 
yhat = clf.predict(X_test)
print("SVM's Accuracy:{0}".format(metrics.accuracy_score(Y_test, yhat)))
acc.append(metrics.accuracy_score(Y_test, yhat))

In [ ]:
Random_forest = RandomForestClassifier(n_estimators=50)
Random_forest.fit(X_train,Y_train)
randomForest_predict = Random_forest.predict(X_test)
randomForest_score = metrics.accuracy_score(Y_test, randomForest_predict)
print("Random Forest Score :",randomForest_score)
acc.append(metrics.accuracy_score(Y_test,randomForest_predict ))

In [ ]:
gbk = GradientBoostingClassifier(random_state=100, n_estimators=150,min_samples_split=100, max_depth=6)
gbk.fit(X_train, Y_train)
gbk_predict = gbk.predict(X_test)
print("Gradient Boosting Score :",metrics.accuracy_score(Y_test,gbk_predict ))
acc.append(metrics.accuracy_score(Y_test,gbk_predict ))

In [ ]:
mx=-1
for i in range(1,25):
    
    neigh=KNeighborsClassifier(n_neighbors = i).fit(X_train,Y_train)
    yhat = neigh.predict(X_test)
    KNN_score=metrics.accuracy_score(Y_test, yhat)
    print("KNN Accuracy at {} is {}".format(i,KNN_score))
    mx=max(mx,KNN_score)
    print("\n")
acc.append(mx)
mx

In [ ]:
algo_name=['Naive Bayes Classifier','Logistic Regression','SVM','Random Forest Classifier','Gradient Boosting','KNN']
acc=np.array(acc)
from numpy import median
plt.figure(figsize=(10,8))
sns.barplot(y=acc*100,x=algo_name,estimator=median,palette="Blues_d")
plt.xlabel('Algorithm Name',size=30)
plt.xticks(rotation=45)
plt.ylabel('Accuracy',size=30)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df.message,df.label,random_state = 0)

In [ ]:
max_features = 4000
maxlen = 50
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)
tokenized_train = tokenizer.texts_to_sequences(x_train)
x_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

# Glove method

In [ ]:
tokenized_test = tokenizer.texts_to_sequences(x_test)
x_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [ ]:
EMBEDDING_FILE = '../input/twitter-embeded/glove.twitter.27B.50d.txt'
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
#change below line if computing normal stats is too slow
embedding_matrix = embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, 50))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
batch_size = 64
epochs = 5
embed_size = 50

In [ ]:
#Defining Neural Network
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(max_features, output_dim=embed_size, weights=[embedding_matrix], input_length = maxlen, trainable=True))
#LSTM 
model.add(Bidirectional(LSTM(units=128, return_sequences = True)))
model.add(Bidirectional(GRU(units=32)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train, batch_size = batch_size , validation_data = (x_test,y_test) , epochs =20)

In [ ]:
print("Accuracy of the model on Testing Data is - " , model.evaluate(x_test,y_test)[1]*100 , "%")

In [ ]:
pred = model.predict_classes(x_test)
pred[:5]

In [ ]:
print(classification_report(y_test, pred, target_names = ['Not Spam','Spam']))

In [ ]:
cm = confusion_matrix(y_test,pred)
cm

In [ ]:
cm = pd.DataFrame(cm , index = ['Not Spam','Spam'] , columns = ['Not Spam','Spam'])

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='' , xticklabels = ['Not Spam','Spam'] , yticklabels = ['Not Spam','Spam'])

# RNN

In [ ]:
df2= pd.read_csv('../input/sms-spam-collection-dataset/spam.csv',delimiter=',',encoding='latin-1')
df2.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df2.info()

In [ ]:
X1= df2.v2
Y1 = df2.v1
le = LabelEncoder()
Y1= le.fit_transform(Y)
Y1 = Y1.reshape(-1,1)

In [ ]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X1,Y1,test_size=0.15)

In [ ]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(Xtrain)
sequences = tok.texts_to_sequences(Xtrain)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
model.fit(sequences_matrix,Ytrain,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

In [ ]:
test_sequences = tok.texts_to_sequences(Xtest)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,Ytest)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
df1

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(df1['clean_msg'], df1['label'], test_size = 0.3, random_state=0, shuffle = True, stratify=df1['label'])

In [ ]:
Xtrain

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def tfidf(data):
    tfidf_vec = TfidfVectorizer()
    tfidf_model = tfidf_vec.fit(data)
    print(tfidf_model.dtype)
    X_tfidf = tfidf_model.transform(data)
    return X_tfidf

In [ ]:
tfidf(Xtrain)

In [ ]:
clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', RandomForestClassifier(n_estimators=100, n_jobs=-1))])

In [ ]:
clf.fit(Xtrain,ytrain)

In [ ]:
! pip install gradio

In [ ]:
import gradio as gr
counter = 0
def prdict(mssg):
    ans=clf.predict([mssg])
    ans=list(ans);ans=ans[0];
    if ans==0:
        s="Ham"
        return s
    else:
        f="Spam"
        return f
iface = gr.Interface(fn=prdict, inputs="text", outputs="text")
iface.launch(share=True)